# &pi; in Julia言語
[Some fun with π in Julia](https://julialang.org/blog/2017/03/piday) ( [Simon Byrne](https://github.com/simonbyrne), [Luis Benet](https://github.com/lbenet), [David P. Sanders](http://sistemas.fciencias.unam.mx/~dsanders/) ) の邦訳です。おつ&pi;！

## &pi; in Julia言語

他の多くの言語と同様、Juliaにも定数&pi;が存在します。 しかし、Juliaでの&pi;の取り扱いは少し特別です。

In [1]:
pi

π = 3.1415926535897...

以下の様に、ユニコード記号で表すこともできます (REPLやJupyter notebookを使用している場合、TeXの様に`\pi`と入力後、Tabキーで自動補完されます)

In [2]:
π

π = 3.1415926535897...

float型変数とは表記が異なることに気が付きましたか？実はJuliaでは&pi;はfloat型ではないのです。

In [3]:
typeof(pi)

Irrational{:π}

&pi;をはじめとする一部の無理数定数は`Float64`型ではなく、`Irrational`型という特殊な型になっています。この型は通常の数値の様に振舞いますが、計算途中で丸めが行われることなく浮動小数点型に変換される特徴を持っています。

In [4]:
1 + pi # 整数型はデフォルトでFloat64型に変換されます

4.141592653589793

In [5]:
Float32(1) + pi # Float32型との演算

4.141593f0

この性質は特に`BigFloat`型と演算する際に役立ちます。`Float64`型に精度が打ち切られる事無く、全桁を評価することができます。

In [6]:
BigFloat(1) + pi # BigFloat型デフォルトの256bitまで計算

4.141592653589793238462643383279502884197169399375105820974944592307816406286198

If &pi; were stored as a `Float64`, we would instead get

もし、&pi;を`Float64`型として扱ってしまうと、以下の様になってしまいます

In [7]:
BigFloat(1) + Float64(pi)

4.141592653589793115997963468544185161590576171875000000000000000000000000000000

`BigFloat`型 ([MPFR](http://www.mpfr.org) libraryを使用しています) は`setprecision`関数を使用することで、任意の精度で計算することができます。

In [8]:
# 1024bitに変換
setprecision(BigFloat, 1024) do 
    BigFloat(pi)
end

3.141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117067982148086513282306647093844609550582231725359408128481117450284102701938521105559644622948954930381964428810975665933446128475648233786783165271201909145648566923460348610454326648213393607260249141273724586997

最後数桁は`BigFloat`型内部でバイナリ形式から十進数形式に変換する際に、多少値が変わってしまう事に注意しましょう。

`Irrational`型のもう一つの便利な特性は、不等号を正確に扱えることでしょう。

In [9]:
Float64(pi) < pi < nextfloat(Float64(pi))

true

## &pi; via インラインアセンブラ


Juliaは低レベルの`llvmcall`インターフェースを実装しています。この機能を使用すると、インラインアセンブラの様な[LLVM中間言語](http://llvm.org/docs/LangRef.html)を直接使用することができます。次の例では、`fldpi`命令 ("Floating point LoaD PI"命令) を使用して、定数&pi;を浮動小数点レジスタにロードします (x86 および x86_64 アーキテクチャでのみ動作します)

In [10]:
function asm_pi()
    Base.llvmcall(
    """ %pi = call double asm "fldpi", "={st}"()
        ret double %pi""", 
    Float64, Tuple{})
end

asm_pi (generic function with 1 method)

In [11]:
asm_pi()

3.141592653589793

実際に生成されるアセンブラコードを見ることもできます。

In [12]:
@code_native asm_pi()

	.text
Filename: In[10]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 2
	pushq	%rax
	fldpi
	fstpl	-8(%rbp)
	movsd	-8(%rbp), %xmm0         # xmm0 = mem[0],zero
	addq	$8, %rsp
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


## テイラー展開による &pi;

ここでは [TaylorSeries.jl](https://github.com/JuliaDiff/TaylorSeries.jl) パッケージを使用し、Taylor展開を用いてを&pi;評価する方法を示します。

In [13]:
# Pkg.add("TaylorSeries")
using TaylorSeries

### Madhavaの公式

三角関数の基本等式に次のような式があります。
$$ \tan\left( \frac{\pi}{6} \right) = \frac{1}{\sqrt{3}}. $$
$6 \arctan(x)$ をテイラー展開し、$1/\sqrt{3}$ を代入すれば $\pi$の値が得られそうですね。実際、この式は収束することが知られています。

`BigFloat`型を使用して、37次までテイラー展開してみましょう。

In [14]:
series1 = 6atan( Taylor1(BigFloat, 37) )
convert(Taylor1{Rational{BigInt}},series1)

 6//1 t - 2//1 t³ + 6//5 t⁵ - 6//7 t⁷ + 2//3 t⁹ - 6//11 t¹¹ + 6//13 t¹³ - 2//5 t¹⁵ + 6//17 t¹⁷ - 6//19 t¹⁹ + 2//7 t²¹ - 6//23 t²³ + 6//25 t²⁵ - 2//9 t²⁷ + 6//29 t²⁹ - 6//31 t³¹ + 2//11 t³³ - 6//35 t³⁵ + 6//37 t³⁷ + 𝒪(t³⁸)

このテイラー展開では奇数乗の項のみ考えればよいので、項が18個しか現れない事に注意しましょう。

$1/\sqrt{3}$についてこの式を評価すると、次の値が得られます。

In [15]:
pi_approx1 = evaluate(series1, 1/sqrt(big(3)))

3.141592653647826046431202390582141253830948237428790668441592864548346569098516

実際の$\pi$との誤差を計算してみましょう。

In [16]:
abs(pi - pi_approx1)

5.803280796855900730263836963377883805368484746664827224053016281231814650118929e-11

より高次のテイラー展開を使えば、より正確な解が得られます。

In [17]:
series2 = 6atan( Taylor1(BigFloat,99) ) # 49個の項が得られます
pi_approx2 = evaluate(series2, 1/sqrt(BigInt(3)))

3.141592653589793238462643347272152237127662423839333289949470742535834074912581

In [18]:
abs(pi - pi_approx2)

3.600735064706950697553577253102547384977198233137361734413175534929622111373249e-26

この公式は [*Madhava級数* または *Gregory–Leibniz級数*](https://en.wikipedia.org/wiki/Madhava_series#Another_formula_for_the_circumference_of_a_circle)として知られています。

\begin{equation}
\pi = 6 \sum_{n=0}^{\infty} (-1)^n \frac{(1/\sqrt{3})^{2n+1}}{2n+1}.
\end{equation}

### Machinのアプローチ

[John Machin](https://ja.wikipedia.org/wiki/%E3%82%B8%E3%83%A7%E3%83%B3%E3%83%BB%E3%83%9E%E3%83%81%E3%83%B3#.E3.83.9E.E3.83.81.E3.83.B3.E3.81.AE.E5.85.AC.E5.BC.8F) は次の等式を使用することで、テイラー展開を使用しながらはるかに早く収束するアルゴリズムを提案しています。

\begin{equation}
\frac{\pi}{4} = 4 \arctan\left(\frac{1}{5}\right) - \arctan\left(\frac{1}{239}\right).
\end{equation}

上記と同様に、37次までテイラー展開してみましょう。

In [19]:
ser = atan( Taylor1(BigFloat, 37) )
pi_approx3 = 4*( 4*evaluate(ser, 1/big(5)) - evaluate(ser, 1/big(239)) )

3.141592653589793238462643383496777424642594661632063407072684671069773618535135

In [20]:
abs(pi - pi_approx3)

2.17274540445425262256957586097740078761957212248936631045983596428448951876822e-28

## Finding guaranteed bounds on &pi;

WIP...

## Summing a series using interval arithmetic

WIP...

## 面積の計算

このセクションでは、もっと簡単な方法で$\pi$を計算してみます。半径$r$の円の面積は $A(r) = \pi r^2$ となるので、 $ r=2$ の円を四等分すると、その面積は$\pi$と等しくなります。

In [21]:
# Pkg.add("Plots")
# Pkg.add("GR")
using Plots; gr();

In [22]:
f(x) = √(4 - x^2)

f (generic function with 1 method)

In [23]:
plot(f, 0, 2, aspect_ratio=:equal, fill=(0, :orange), alpha=0.2, label="")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
<polygon clip-path="url(#clip02)" points="
125.283,3.93701 159.69,5.83128 180.367,8.48391 197.849,11.6338 216.84,16.0233 234.347,20.9937 253.117,27.3498 274.864,36.1189 294.191,45.2676 311.138,54.4265 
 329.615,65.7341 348.202,78.6508 368.752,94.9811 388.305,112.86 408.636,134.403 425.75,155.483 436.197,170.031 446.644,186.187 456.465,203.221 466.285,222.62 
 474.152,240.5 482.019,261.433 485.952,273.57 489.885,287.362 493.819,303.645 497.752,324.397 498.963,332.468 500.173,342.064 501.384,354.602 501.384,384.952 
 500.173,384.952 498.963,384.952 497.752,384.952 493.819,384.952 489.885,384.952 485.952,384.952 482.019,384.952 474.152,384.952 466.285,384.952 456.465,384.952 
 446.644,384.952 436.197,384.952 425.75,384.952 408.636,384.952 388.305,384.952 368.752,384.952 348.202,384.952 329.615,384.952 311.138,384.952 294.191,384.952 
 274.864,384.952 253.117,384.952 234.347,384.952 216.84,384.952 197.849,384.952 180.367,384.952 159.69,384.952 125.283,384.952 
 " fill="#ffa500" fill-opacity="0.2"/>
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:0.2; fill:none" points="
 125.283,3.93701 159.69,5.83128 180.367,8.48391 197.849,11.6338 216.84,16.0233 234.347,20.9937 253.117,27.3498 274.864,36.1189 294.191,45.2676 311.138,54.4265 
 329.615,65.7341 348.202,78.6508 368.752,94.9811 388.305,112.86 408.636,134.403 425.75,155.483 436.197,170.031 446.644,186.187 456.465,203.221 466.285,222.62 
 474.152,240.5 482.019,261.433 485.952,273.57 489.885,287.362 493.819,303.645 497.752,324.397 498.963,332.468 500.173,342.064 501.384,354.602 
 "/>

The circle of radius $r=2$ is given by $x^2 + y^2 = 2^2 = 4$, so 

半径 $r=2$ の円は $x^2 + y^2 = 2^2 = 4$ と表せるため、$\pi$ は

$$\pi = \frac{1}{4} A(2) = \int_{x=0}^2 y(x) \, dx = \int_{x=0}^2 \sqrt{4 - x^2}.$$

となります。

微分積分学の応用として、積分を**リーマン和**を用いて近似してみましょう。区間演算を使用すると、以下の様にリーマン和をシンプルかつ厳密に表現することができます。

試しにx軸を等間隔で分割してみましょう。

In [24]:
#Pkg.add("ValidatedNumerics")
using ValidatedNumerics

function make_intervals(N=10)
    xs = linspace(0, 2, N+1)
    return [xs[i]..xs[i+1] for i in 1:length(xs)-1]
end

intervals = make_intervals()

10-element Array{ValidatedNumerics.Interval{Float64},1}:
 [0, 0.200001]          
    [0.199999, 0.400001]
    [0.399999, 0.600001]
    [0.599999, 0.800001]
    [0.799999, 1]       
 [1, 1.20001]           
    [1.19999, 1.40001]  
    [1.39999, 1.60001]  
    [1.59999, 1.80001]  
    [1.79999, 2]        

ある区間の関数の値は以下の様にして得られます。

In [25]:
II = intervals[1]

[0, 0.200001]

In [26]:
f(II)

[1.98997, 2]

ある区間における関数値の下限上限は、リーマン積分の箱の下限上限に対応しています。


In [27]:
intervals = make_intervals(30)

p = plot(aspect_ratio=:equal)
for X in intervals
    Y = f(X)
    
    plot!(IntervalBox(X, Interval(0, Y.lo)), c=:blue, label="", alpha=0.1)
    plot!(IntervalBox(X, Interval(Y.lo, Y.hi)), c=:red, label="", alpha=0.1)
end

plot!(f, 0, 2)

p

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 126.547,3.95518 160.952,5.84936 181.628,8.50187 199.11,11.6516 218.099,16.0409 235.606,21.0111 254.375,27.3669 276.121,36.1356 295.447,45.2838 312.393,54.4423 
 330.869,65.7493 349.455,78.6654 370.005,94.9949 389.557,112.873 409.886,134.415 427,155.494 437.446,170.042 447.893,186.196 457.713,203.23 467.533,222.627 
 475.399,240.507 483.266,261.439 487.199,273.576 491.132,287.367 495.065,303.649 498.998,324.4 500.209,332.47 501.42,342.067 502.63,354.603 
 "/>
 
 
 
 
 y61

早速、箱の面積の和を計算してみましょう。

In [28]:
N = 20
intervals = make_intervals(N)

width = 2/N
width * sum(√(4 - X^2) for X in intervals)

[3.02846, 3.22847]

区間の分割数を増やすにつれ、近似がより正確になっていきます。

In [29]:
#setdisplay(:standard, sigfigs=5)

println("N \t area interval \t \t diameter")
for N in 50:50:1000
    intervals = make_intervals(N)
    area = (2/N) * sum(√(4 - X^2) for X in intervals)
            
    println("$N \t $area \t $(diam(area))")
end

N 	 area interval 	 	 diameter
50 	 [3.09826, 3.17827] 	 0.0800000000000165
100 	 [3.12041, 3.16042] 	 0.040000000000032454
150 	 [3.12761, 3.15429] 	 0.02666666666670814
200 	 [3.13117, 3.15118] 	 0.02000000000006308
250 	 [3.13329, 3.1493] 	 0.016000000000075065
300 	 [3.13469, 3.14804] 	 0.013333333333415354
350 	 [3.13569, 3.14713] 	 0.011428571428676815
400 	 [3.13644, 3.14645] 	 0.010000000000123688
450 	 [3.13702, 3.14592] 	 0.008888888889027502
500 	 [3.13748, 3.14549] 	 0.008000000000148333
550 	 [3.13786, 3.14514] 	 0.007272727272884527
600 	 [3.13817, 3.14485] 	 0.006666666666829357
650 	 [3.13844, 3.1446] 	 0.006153846154013376
700 	 [3.13867, 3.14439] 	 0.0057142857144931725
750 	 [3.13886, 3.14421] 	 0.005333333333562784
800 	 [3.13904, 3.14405] 	 0.005000000000246363
850 	 [3.13919, 3.1439] 	 0.004705882353203794
900 	 [3.13932, 3.14378] 	 0.004444444444719142
950 	 [3.13944, 3.14366] 	 0.004210526316076102
1000 	 [3.13955, 3.14356] 	 0.004000000000294435


Original post is [Some fun with π in Julia](https://julialang.org/blog/2017/03/piday) ( [Simon Byrne](https://github.com/simonbyrne), [Luis Benet](https://github.com/lbenet), [David P. Sanders](http://sistemas.fciencias.unam.mx/~dsanders/) ).


> Copyright (c) 2009-2016: Jeff Bezanson, Stefan Karpinski, Viral B. Shah,
> and other contributors:
>
> https://github.com/JuliaLang/julia/contributors
>
> Permission is hereby granted, free of charge, to any person obtaining
> a copy of this software and associated documentation files (the
> "Software"), to deal in the Software without restriction, including
> without limitation the rights to use, copy, modify, merge, publish,
> distribute, sublicense, and/or sell copies of the Software, and to
> permit persons to whom the Software is furnished to do so, subject to
> the following conditions:
>
> The above copyright notice and this permission notice shall be
> included in all copies or substantial portions of the Software.
>
> THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
> EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
> MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
> NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE
> LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION
> OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION
> WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.